# Discoveryでコレクション作成

In [ ]:
# 個別設定
# 管理コンソールのcredentail情報をコピーします。
credencial = {
  "url": "https://gateway.watsonplatform.net/discovery/api",
  "username": "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
  "password": "xxxxxxxxxxxx"
}

In [ ]:
# ライブラリ読み込み
import json
from watson_developer_cloud import DiscoveryV1
# In the constructor, letting the SDK manage the IAM token
discovery = DiscoveryV1(version = '2018-03-05',
                        username = credencial['username'],
                       password = credencial['password'])

In [ ]:
# Environment作成
response = discovery.create_environment(
    name="my_environment"
)
# environment_idの設定
environment_id = response['environment_id']
print(environment_id)

In [ ]:
# Collection作成
col_name = 'JP_TEST'
new_collection = discovery.create_collection(
    environment_id = response['environment_id'], 
    name = col_name, 
    language='ja')

# collection_idの設定
collection_id = new_collection['collection_id']
print(collection_id)

In [ ]:
# 必要であれば、このタイミングでエンリッチ設定の変更を行います。
# 一度コレクションを日本語で作ってしまえば、エンリッチ設定変更はUIから可能です。

# CSVファイル読込み

In [ ]:
infile = 'data_xxxxxx.csv'

# このセルは、ツール生成のものを一部手で修正します。

import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

client_xxxxxx = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xxxxxx',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

# Key=のパラメータは変数利用に修正しました
body = client_38bc5a04c29444d093d357a963ea4f0d.get_object(Bucket='xxxxxxx',Key=infile)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# DataFrame名は df_dataに修正しました
df_data = pd.read_csv(body)
df_data.head()

# 読み込んだデータをDiscovey Collectionに書き込み

In [ ]:
import os
for i in range(len(df_data)):
    item = df_data.iloc[i,:]
    data = {}
    
    # 以下はCSVの項目名によって修正して下さい
    data['pat_id'] = str(item['出願番号'])
    data['pat_name'] = item['発明の名称']
    data['summury'] = item['要約']
    data['text'] = item['請求の範囲']
    # print(data)
    filename = data['pat_id'] + '.json'
    f = open(filename, 'w')
    json.dump(data, f)
    f.close()
    collection = discovery.get_collection(environment_id, collection_id)
    # print(json.dumps(collection, indent=2))
    with open(filename) as f:
        add_doc = discovery.add_document(environment_id, collection_id, file = f)
    os.remove(filename)
    # print(json.dumps(add_doc, indent=2))

# 検索テスト

In [ ]:
query = '検索テスト'
filter = '検索テスト'

In [ ]:
my_query = discovery.query(
    environment_id = environment_id, 
    collection_id = collection_id, 
    query = query, 
    filter = filter, 
    return_fields = '['pat_id', 'pat_name']')
print(json.dumps(my_query, indent=2))

# データ削除用 (MAX100件)

In [ ]:
results = discovery.query(environment_id, collection_id, return_fields='id', count=100)["results"]
ids = [item["id"] for item in results]

for id in ids:
    print('deleting doc: id =' + id)
    discovery.delete_document(environment_id, collection_id, id)